# Compute Radiomics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/Radiomics.ipynb)

In this example notebook we use [PyRadiomics](https://github.com/AIM-Harvard/pyradiomics) to
compute various type of radiomics features. We use some
[LCTSC](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24284539) data from the
Cancer Imaging Archive which has already been converted using PyDicer for demonstration purposes.

In [1]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.utils import fetch_converted_test_data

from pydicer.utils import load_object_metadata

## Fetch data

LCTSC data prepared for this example are downloaded and stored into a `testdata_lctsc` directory.
We will use this for our PyDicer working directory.

In [2]:
working_directory = fetch_converted_test_data("./testdata_lctsc", dataset="LCTSC")

Working directory %s aready exists, won't download test data.


## Initialise PyDicer object

Using the working directory containing the LCTSC test data.

In [3]:
pydicer = PyDicer(working_directory)

## Compute Default Radiomics

By default, PyDicer will compute only first-order radiomics features. Radiomics are computed for
each structure available in the dataset using the image data of images linked to those structures.

In [4]:
pydicer.analyse.compute_radiomics()

pydicer.analyse.data - INFO - Computing radiomics for structure set ed6686 and image 5adf40


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_L


pydicer.analyse.data - DEBUG - Computing radiomics for structure Esophagus


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_R


pydicer.analyse.data - DEBUG - Computing radiomics for structure SpinalCord


...

pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_L


pydicer.analyse.data - DEBUG - Computing radiomics for structure Esophagus


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_R


pydicer.analyse.data - DEBUG - Computing radiomics for structure SpinalCord


pydicer.analyse.data - DEBUG - Computing radiomics for structure Heart


## Fetch computed Radiomics

Use the [get_all_computed_radiomics_for_dataset](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.get_all_computed_radiomics_for_dataset) function to fetch all radiomics features computed
in the last step.

The `.head()` function on a Pandas DataFrame output the first 5 rows for inspection.

In [5]:
# Display the DataFrame of radiomics computed
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


...

pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


,Contour,Patient,ImageHashedUID,StructHashedUID,ResampledPixelSpacing,NormalisationScale,firstorder|10Percentile,firstorder|90Percentile,firstorder|Energy,firstorder|Entropy,...,firstorder|Mean,firstorder|Median,firstorder|Minimum,firstorder|Range,firstorder|RobustMeanAbsoluteDeviation,firstorder|RootMeanSquared,firstorder|Skewness,firstorder|TotalEnergy,firstorder|Uniformity,firstorder|Variance
1,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-9.0,74.0,6.379739e+06,2.563380,...,33.382764,42.0,-745.0,940.0,15.951183,70.379070,-5.961993,1.642731e+08,0.232723,3838.804579
4,Heart,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-14.0,68.0,8.486142e+07,2.343017,...,34.078823,44.0,-577.0,806.0,14.250737,53.881633,-2.753651,2.185112e+09,0.273513,1741.864194
0,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-871.0,-502.0,3.907161e+10,4.313815,...,-729.756106,-778.0,-1000.0,1357.0,67.906500,747.249117,1.747786,1.006062e+12,0.069585,25837.268599
2,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-889.0,-480.0,4.790689e+10,4.414603,...,-735.842185,-789.0,-1000.0,1215.0,76.876692,755.465238,1.617379,1.233563e+12,0.063675,29264.004413
3,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-19.0,70.0,6.830546e+06,2.638360,...,27.051730,27.0,-144.0,573.0,18.213519,49.364661,1.348291,1.758810e+08,0.205445,1705.073671


## Specify Radiomics to Compute

PyDicer used the popular `pyradiomics` library to compute radiomics. So, you may specify any
radiomics features provided in that library to be computed. See the [pyradiomics documentation for
a list of radiomics features
available](https://pyradiomics.readthedocs.io/en/latest/features.html).

In this example, we specify all `shape` features as well as `first-order` features to be computed.

In [6]:
# Import pyradiomics feature modules
from radiomics import (
    firstorder,
    shape,
)

# Prepare a dict of features to compute grouped by class
first_order_features = firstorder.RadiomicsFirstOrder.getFeatureNames()
shape_features = shape.RadiomicsShape.getFeatureNames()
compute_radiomics = {
    "firstorder": [f for f in first_order_features if not first_order_features[f]],
    "shape": [f for f in shape_features if not shape_features[f]],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

# Fetch the computed radiomics and output the first few rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

pydicer.analyse.data - INFO - Computing radiomics for structure set ed6686 and image 5adf40


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_L


pydicer.analyse.data - DEBUG - Computing radiomics for structure Esophagus


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_R


pydicer.analyse.data - DEBUG - Computing radiomics for structure SpinalCord


...

pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


,Contour,Patient,ImageHashedUID,StructHashedUID,ResampledPixelSpacing,NormalisationScale,firstorder|10Percentile,firstorder|90Percentile,firstorder|Energy,firstorder|Entropy,...,shape|Maximum2DDiameterColumn,shape|Maximum2DDiameterRow,shape|Maximum2DDiameterSlice,shape|Maximum3DDiameter,shape|MeshVolume,shape|MinorAxisLength,shape|Sphericity,shape|SurfaceArea,shape|SurfaceVolumeRatio,shape|VoxelVolume
1,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-9.0,74.0,6.379739e+06,2.563380,...,223.924778,222.309084,23.619884,224.231209,3.193220e+04,50.733471,0.408351,11919.810387,0.373285,3.316495e+04
4,Heart,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-14.0,68.0,8.486142e+07,2.343017,...,163.733306,127.040344,151.241053,164.152138,7.510135e+05,100.977462,0.781597,51121.049949,0.068069,7.526486e+05
0,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-871.0,-502.0,3.907161e+10,4.313815,...,236.649430,242.714238,190.429595,245.982024,1.802886e+06,171.702531,0.568696,125964.905979,0.069868,1.801747e+06
2,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-889.0,-480.0,4.790689e+10,4.414603,...,231.159117,247.961931,186.213760,248.126635,2.159708e+06,172.513299,0.582917,138614.093158,0.064182,2.161386e+06
3,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,-19.0,70.0,6.830546e+06,2.638360,...,300.128718,313.974459,26.529436,314.315983,7.050233e+04,32.966235,0.418454,19722.946105,0.279749,7.217496e+04


You can also set a specific subset of features like this:

In [7]:
# Specify features to compute grouped by class
compute_radiomics = {
    "firstorder": ["Maximum", "Minimum", "Mean", "Median"],
    "shape": ["SurfaceArea", "VoxelVolume"],
}

# Pass the dict to the compute the radiomics
pydicer.analyse.compute_radiomics(radiomics=compute_radiomics)

df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

pydicer.analyse.data - INFO - Computing radiomics for structure set ed6686 and image 5adf40


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_L


pydicer.analyse.data - DEBUG - Computing radiomics for structure Esophagus


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_R


pydicer.analyse.data - DEBUG - Computing radiomics for structure SpinalCord


...

pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


,Contour,Patient,ImageHashedUID,StructHashedUID,ResampledPixelSpacing,NormalisationScale,firstorder|Maximum,firstorder|Minimum,firstorder|Mean,firstorder|Median,shape|SurfaceArea,shape|VoxelVolume
1,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,195.0,-745.0,33.382764,42.0,11919.810387,3.316495e+04
4,Heart,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,229.0,-577.0,34.078823,44.0,51121.049949,7.526486e+05
0,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,357.0,-1000.0,-729.756106,-778.0,125964.905979,1.801747e+06
2,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,215.0,-1000.0,-735.842185,-789.0,138614.093158,2.161386e+06
3,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,429.0,-144.0,27.051730,27.0,19722.946105,7.217496e+04


## Track Metadata

When analysing your radiomic features, it may be useful to have certain metadata available from
either the image or structure set. You can specify which DICOM header tags to extract metadata for
and these will be stored alongside the radiomic feature values.

In the cell below, we recompute our radiomics and store the `PatientSex` header value from the
image series and the `StudyDate` value from the structure set.

In [8]:
# Compute the radiomics specifying the meta data to keep
pydicer.analyse.compute_radiomics(
    radiomics=compute_radiomics,
    image_meta_data=["PatientSex"],
    structure_meta_data=["StudyDate"]
)

# Fetch the results and display the first rows
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset()
df_radiomics.head()

pydicer.analyse.data - INFO - Computing radiomics for structure set ed6686 and image 5adf40


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_L


pydicer.analyse.data - DEBUG - Computing radiomics for structure Esophagus


pydicer.analyse.data - DEBUG - Computing radiomics for structure Lung_R


pydicer.analyse.data - DEBUG - Computing radiomics for structure SpinalCord


...

pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


pydicer.dataset.structureset - WARNING - No mapping file found with id default


,Contour,Patient,ImageHashedUID,StructHashedUID,ResampledPixelSpacing,NormalisationScale,firstorder|Maximum,firstorder|Minimum,firstorder|Mean,firstorder|Median,shape|SurfaceArea,shape|VoxelVolume,struct|StudyDate,img|PatientSex
1,Esophagus,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,195.0,-745.0,33.382764,42.0,11919.810387,3.316495e+04,20040303,M
4,Heart,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,229.0,-577.0,34.078823,44.0,51121.049949,7.526486e+05,20040303,M
0,Lung_L,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,357.0,-1000.0,-729.756106,-778.0,125964.905979,1.801747e+06,20040303,M
2,Lung_R,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,215.0,-1000.0,-735.842185,-789.0,138614.093158,2.161386e+06,20040303,M
3,SpinalCord,LCTSC-Test-S1-101,666be6,cc682f,NaN,NaN,429.0,-144.0,27.051730,27.0,19722.946105,7.217496e+04,20040303,M
